# Imports

In [ ]:
import wandb
import pandas as pd

# Guide

See here for guide https://docs.wandb.ai/guides/track/public-api-guide

# Single run

In [ ]:
run_id = "kn9o0vjo"
api = wandb.Api()
run = api.run("philipwastaken/02456_project/runs/" + run_id)
history = run.scan_history()

In [ ]:
train_dict = {'mean_train_reward':[],
            'frame_count': [],
            '_timestamp': []}

validate_dict = {'sims_per_val':[],
               'avg_reward':[],
               'duration':[],
               'avg_frame_count':[]}

for row in history:
    if 'validate/sims_per_val' in row:
        validate_dict['sims_per_val'].append(row['validate/sims_per_val'])
        validate_dict['avg_reward'].append(row['validate/avg_reward'])
        validate_dict['duration'].append(row['validate/duration'])
        validate_dict['avg_frame_count'].append(row['validate/avg_frame_count'])
    else:
        train_dict['mean_train_reward'].append(row['mean_train_reward'])
        train_dict['frame_count'].append(row['frame_count'])
        train_dict['_timestamp'].append(row['_timestamp'])
        
train_df = pd.DataFrame(train_dict)
validate_df = pd.DataFrame(validate_dict)

In [ ]:
train_df

In [ ]:
validate_df

# Multiple Runs

In [38]:
api = wandb.Api() 
runs = api.runs("philipwastaken/02456_project")

In [39]:
for run in runs: 
    print(run)

<Run philipwastaken/02456_project/2iomhs4l (running)>
<Run philipwastaken/02456_project/4tzwvexw (running)>
<Run philipwastaken/02456_project/31k5tjl1 (running)>
<Run philipwastaken/02456_project/y4wrkyh6 (running)>
<Run philipwastaken/02456_project/3ebfslui (crashed)>
<Run philipwastaken/02456_project/28rdoyn5 (running)>
<Run philipwastaken/02456_project/22q76qoq (running)>
<Run philipwastaken/02456_project/1do6x5bd (running)>
<Run philipwastaken/02456_project/28hz8kw0 (running)>
<Run philipwastaken/02456_project/1nct5vni (running)>
<Run philipwastaken/02456_project/3j2umlgo (finished)>
<Run philipwastaken/02456_project/2575mge7 (finished)>
<Run philipwastaken/02456_project/131sqsih (running)>
<Run philipwastaken/02456_project/2gwfsl42 (finished)>
<Run philipwastaken/02456_project/5hsv2744 (finished)>
<Run philipwastaken/02456_project/3ptcsscu (crashed)>
<Run philipwastaken/02456_project/15uf3r8n (finished)>
<Run philipwastaken/02456_project/h86ys382 (finished)>
<Run philipwastaken/02